# Error propagation
Script to test the method for propagating errors on the brightness temperature calculation.   
Baxter et al 2020 steps: 
- Calculates the 1 sigma minimum and maximum eclipse depth as (Fp/Fs - Fp/Fs_err) and (Fp/Fs + Fp/Fs_err)  
- Calculates the brightness temperatures of these minimum and maximum eclipse depths  
- Calculates the upper and lower error on Tb by doing (Tb_err_max - Tb) and (Tb - Tb_err_min)  

Garhart et al 2020 - seems to assume RJ approx or just propagates the percentage error..

In [63]:
from sympy import Symbol, Derivative,log, diff, sqrt
from astropy import units as u
from astropy import constants as const
import numpy as np

## Set up the problem
Integrating Equation 1 from Baxter et al to find the errors on brightness temperature

In [60]:
# Parameters "without" errors 
h = Symbol('h', constant=True, number=True)
c = Symbol('c', constant=True, number=True)
k = Symbol('k', constant=True, number=True)
lam = Symbol('lam', constant=True, number=True)
Fs = Symbol('Fs', constant=True, number=True)

# Parameters with errors
rprs = Symbol('rprs')
fpfs = Symbol('fpfs')

# Inverted Planck functions in terms of transit depth, eclipse depth and Fs (Equation 1 of Baxter et al 2020)
Tp = (((h*c) / (k*lam) ) * 1./
        (log(((2.*h*c**2*np.pi*rprs**2)/
        (lam**5*Fs*fpfs)) + 1. )))

Propagate the equation for the brightness temperature using this simplification:  


$s_{f}={\sqrt {\left({\frac {\partial f}{\partial x}}\right)^{2}s_{x}^{2}+\left({\frac {\partial f}{\partial y}}\right)^{2}s_{y}^{2}+\left({\frac {\partial f}{\partial z}}\right)^{2}s_{z}^{2}+\cdots }}$  

Where we are interested in the error on the eclipse depth (Fp/Fs) and also maybe the transit depth (Rp/Rs) 

## Differentiate $Tp$ to plug into $s_f$

In [83]:
# Differentiate wrt eclipse depth and transit depth
dTpdrprs = diff(Tp,rprs)
dTpdfpfs = diff(Tp,fpfs)

# Initiate symbols for the errors
rprs_err = Symbol('rprs_err', constant=True, number=True)
fpfs_err = Symbol('fpfs_err', constant=True, number=True)

# Create the equation for the brightness temperature error
sigmaTp = sqrt( (dTpdrprs * rprs_err)**2 + (dTpdfpfs * fpfs_err)**2)

## Plug in the values for Hat-p-13b to calculate the error  
Rp/Rs = 0.08389±0.00081  
Fp/Fs = 851±107 ppm  
lam = 3.6e-6 m  
Fp = 7521 erg/cm^2/s/Angstrom  
^ Fp is an output of my code when calculating brightness temperature. It involves integration of PHOENIX models over Spitzer bandpass for the stellar flux (Fs), which is then converted to Fp using the eclipse depth (Fp/Fs) and Rp/Rs and returned as an output of my code. Here I take that output and jankily convert back to Fs by diving by the eclipse depth and Rp/Rs again.

In [100]:
print "{:.2f} % error on Rp/Rs".format(100*0.00081/0.08389)
print "{:.2f} % error on Fp/Fs".format(100*107/851)

fluxscale = (u.erg/(u.cm**2*u.s*u.Angstrom)).to(u.J/u.s/u.m**3)

# Substitute in all of the values for Hatp-13b
TpH13 = Tp.subs({h:const.h.value, c:const.c.value, k:const.k_B.value, 
                   fpfs:851e-6, # 851 ppm
                   Fs:7520./851e-6*fluxscale*(0.08389**2), 
                   rprs:0.08389, # Rp/Rs
                   lam:3.6e-6}) # 3.6 micron

sigmaTpH13 = sigmaTp.subs({h:const.h.value, c:const.c.value, k:const.k_B.value, 
                           fpfs:851e-6, # 851 ppm
                           Fs:7521./851e-6*fluxscale*(0.08389**2), # Fp =7520 erg/cm^2/s/Angstrom to J/s/m^3. Invert Eclipse depth and multiply by Rp/Rs^2
                           rprs:0.08389, # Rp/Rs
                           lam:3.6e-6, # 3.6 micron
                           fpfs_err:107e-6, rprs_err: 0.00081}) 

print r"Brightness Temperature: {:.0f} +/- {:.0f} K @ 3.6um".format(TpH13, sigmaTpH13)
print "{:.2f} % error on Tb".format(100*sigmaTpH13/TpH13)

0.97 % error on Rp/Rs
12.00 % error on Fp/Fs
Brightness Temperature: 1798 +/- 92 K @ 3.6um
5.10 % error on Tb


#### Brightness temperature error of 92 Kelvin is in agreement with the method used in Baxter et al 2020 (Tb = 1775 ± 87 K) compared to the value in Garhart et al 2020 (Tb = 1810 ± 229 K)